In [1]:
import os
%pwd

'f:\\ml projects\\chicken_disease_classification\\notebooks'

In [2]:
os.chdir('../')
%pwd

'f:\\ml projects\\chicken_disease_classification'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_size:list
    params_batch_size:int


In [6]:
from chicken_disease_classifier.constants import *
from chicken_disease_classifier.utils.common import create_directories,read_yaml,save_json

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> ModelEvaluationConfig:
        evaluation_config = ModelEvaluationConfig(
            path_of_model= Path(self.config.training.trained_model_path),
            training_data= Path(os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")),
            all_params=self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
            
        )
        return evaluation_config

In [8]:
from urllib.parse import urlparse

In [14]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig) -> None:
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.models:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {'loss':self.score[0],'accuracy':self.score[1]}
        save_json(path= Path("scores.json"),data=scores)

In [15]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_validation_config()
    evaluate_model = ModelEvaluation(config=evaluation_config)
    evaluate_model.evaluation()
    evaluate_model.save_score()
except Exception as e:
    raise e

[ 2023-11-23 16:07:50,590 ] 26 - common - INFO - <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-23 16:07:50,596 ] 26 - common - INFO - <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-23 16:07:50,599 ] 48 - common - INFO - Created Directory at : artifacts


Found 78 images belonging to 2 classes.
5/5 [==============================] - 35s 7s/step - loss: 0.4424 - accuracy: 0.9103
[ 2023-11-23 16:08:27,240 ] 61 - common - INFO - Json file saved at : scores.json
